<a href="https://colab.research.google.com/github/chefs-kiss/ML_J2026/blob/main/PA4_Data_Cleaning_with_AirBnB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Name:

Who you worked with:


## Objectives
The goals of this project are to:
- Perform EDA and data cleaning
- Implement transformations and filtering functions
- Look for EDA and data cleaning inspiration from outside sources

## Overview
For this programming assignment, you will practice some data cleaning and preprocessing, using a dataset of AirBNB data from New York City. In working with this dataset, our goal will be to train a model that can predict the price of an AirBNB rental. Imagine that you are a data scientist working for AirBNB, and your boss has asked you to develop this as a tool that can suggest prices for new listings.

## Schedule
Here is the suggested schedule for working on this project:
- Weekend: Read through project instructions, complete Task 1.
- Tuesday: Complete Tasks 2-3.
- Wednesday: Complete Tasks 4-5.
- Thursday: Complete Task 6.

This project is due on Thursday, 3/13, by 11:59pm.


# Task 1: NYC Data

The data is available from a dataset from OpenML (among many other places) that contains information on multiple cities.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_openml
airbnb = fetch_openml(name="U.S.-Airbnb-Open-Data", as_frame=True)
X = airbnb['data']
y = airbnb['target']
airbnb_df = pd.concat([X, y], axis=1)

##💻 Q1: NYC Data Only

Your first task is to create a subset dataframe called `nyc` that filters down to only records where the city is New York City.

The format for this is:
```
selection = df[col] == thing
subset_df = df[selection]
```
or you can condense this down to
```
subset_df = df[df[col] == thing]
```
where `thing` is what you're filtering on, and `df` is the original dataframe

You will need to update the code chunk below and replace `df`, `col` and `thing` with the appropriate information from our dataset. Keep the `.copy(deep=True)` chained at the end. Once you're satisfied with your selections, make sure to uncomment out the line of code and run it.

In [ ]:
nyc = #df[df[col] == thing].copy(deep=True)

Let's look at the first few rows and the column names.

In [ ]:
nyc.head(5)

In [ ]:
nyc.columns

##✏ Q2: Information on Columns

For each column in the dataset, write a brief description (1-2 sentences) of what it represents. There is a helpful [data dictionary](https://docs.google.com/spreadsheets/d/1b_dvmyhb_kAJhUmv81rAxl4KcXn0Pymz/edit?gid=1967362979#gid=1967362979) that is referenced on this [Kaggle site](https://www.kaggle.com/datasets/arianazmoudeh/airbnbopendata) for the dataset. Use either to help you with this step.

[your answers here]

## Held-Out Set
Now, I'm going to select a few records that we'll use at the end to test a prediction function. I'm selecting these here, since we'll be making changes to the dataset later.

In [ ]:
chosen_indices = range(0, 9000, 1000)

held_out = nyc.iloc[chosen_indices]

held_out

# Task2: Dropping Features

Now, we're going to drop some of the columns from the dataset. We start by listing all of the columns from the dataframe.

In [ ]:
nyc.columns

We drop some of the columns here. The format is

```
df.drop(cols_to_drop, axis = 1, inplace = True)
```
where `df` is our dataframem, and `cols_to_drop` are the columns we decided to remove.

In [ ]:
cols_to_drop = ['id', 'name', 'host_id', 'host_name', 'minimum_nights', 'last_review', 'reviews_per_month', 'availability_365']
nyc.drop(columns = cols_to_drop, axis = 1, inplace = True)

In [ ]:
nyc.columns

##✏ Q3: Dropping Features
For each dropped column, explain why your think it is a reasonable or unreasonable choice to drop that column. Feel free to disagree with my choices! (Though we'll stick with my choices for this assignment.)

[your answers here]

Now, let's look at the columns we have left

In [ ]:
nyc.columns

##✏ Q4: Explain reasoning

For each remaining column, explain why your think it is a reasonable or unreasonable choice to keep that column. Feel free to disagree with my choices!

[your answers here]

Now, let's look at what we have left in our data.

In [ ]:
nyc.describe(include = "all")

# Task 3: Numeric Features
Our data cleaning and preprocessing will focus on transformations to individual features.

## Price

 We'll start by looking at our target, price. Let's look at a histogram, to see how the prices are distributed.

In [ ]:
nyc['price'].hist(bins = 80)

This is extremely skewed. We have a price that's \$100,000, but the vast majority have a price under \$1000.

Thinking about the tool we're building, we might want to focus on training a model that predicts well for lower/typical prices, and ignore the really expensive ones. Let's look at what happens with a few different choices for restricting the prices.

First, we look at prices below $2,000.

In [ ]:
print("Number of entries: ", len(nyc[nyc['price'] < 2000]))

nyc[nyc['price'] < 2000]['price'].hist(bins = 100)

Next, let's look at prices below $1,000.

In [ ]:
print("Number of entries: ", len(nyc[nyc['price'] < 1000]))

nyc[nyc['price'] < 1000]['price'].hist(bins = 100)

Here, we look at prices below $500.

In [ ]:
print("Number of entries: ", len(nyc[nyc['price'] < 500]))

nyc[nyc['price'] < 500]['price'].hist(bins = 100)

Let's focus on prices below $500. This still captures a lot of the data, while limiting us to trying to predict prices for more typical listings. Note that the distribution is still skewed - we'll handle this in a bit.

In [ ]:
nyc = nyc[nyc['price'] < 500]

nyc.describe(include = "all")

Notice that the minimum price is \$0, which seems like nonsense. Let's look at the listings with a price of \$0.

In [ ]:
nyc[nyc['price'] <= 0]

There aren't too many of them, so let's just exclude those.

In [ ]:
nyc = nyc[nyc['price'] > 0]

nyc.describe(include = "all")

We've seen that even restricting to prices below $500, we still have a skewed distribution. Let's see if applying a transformation can help. First, we try applying a log function.

In [ ]:
nyc["price"].apply(np.log).hist(bins = 50)

Let's also see what happens with applying a square root function.

In [ ]:
nyc["price"].apply(np.sqrt).hist(bins = 50)

We'll choose to apply the log transformation. Note that applying such a transformation does make sense here, because the difference between prices \$400 and \$450 is less significant than the difference between prices \$50 and \$100.

##💻 Q5: Log Price
Add a new column called `log_price`, which has the log transformation appied to the column `price`. Then, drop `price` from the dataframe.

In [ ]:
# add column log_price
nyc["log_price"] = #your code here

#drop price
#your code here

nyc.describe(include = "all")

In [ ]:
assert(np.isclose(nyc["log_price"].mean(), 4.60618))
assert("price" not in nyc.columns)

## Latitude and Longitude

Now, let's take a look at latitude and longitude. We'll look at the maximum and mininum values, to see the range they fall in.

In [ ]:
print(nyc['latitude'].max())
print(nyc['latitude'].min())
print(nyc['longitude'].max())
print(nyc['longitude'].min())

This range makes sense here, since all of the listings are in New York City. Now, let's look at histograms.

In [ ]:
nyc['latitude'].hist(bins = 50)

In [ ]:
nyc['longitude'].hist(bins = 50)

The distributions look pretty close to normal, so we'll leave them as they are.

## Number of Reviews

Now, let's look at number of reviews. We'll create a histogram to see the distribution of the data.

In [ ]:
nyc['number_of_reviews'].hist(bins = 50)

This has a very dramatic right skew, so let's try applying some transformations. We'll start with a log transformation.

Now, if we were to apply the log transformation to this feature we would end up with an error. To fix this we need to add 1 to each value prior to log transforming it. This process is called smoothing and can be particularly helpful when working with text data in the future.

##✏ Q6: Add one smoothing

Explain why we need to add 1. Hint: are there any values that when you apply the log transformation you would run into issues?


[your answer here]

Now let's visualize this add-one smoothing log transformation

In [ ]:
nyc["number_of_reviews"].apply(lambda x: (np.log(x+1))).hist(bins = 50)

This is an improvement, but still right skewed. Let's look at a square root transformation next.

In [ ]:
nyc["number_of_reviews"].apply(np.sqrt).hist(bins = 50)

That's not as good as the log transformation.

Since the result of the log transformation is still pretty skewed, let's try applying the log transformation again. Again, we'll have to add 1 before applying the log transformation.

##💻 Q7: log-log function
In the cell below, define a function called `log_log` that does the following computation:
$$ f(x) = \log(\log(x+1)+1)$$.

Note: replace the keyword `pass` with the computation above.

In [ ]:
def log_log(value):
  pass

# create histogram
nyc["number_of_reviews"].apply(log_log).hist(bins = 50)

This still isn't great, but we can see that we're going to have trouble improving much more, since the number of reviews is going to be very discrete for small numbers (hence the tall, isolated columns on the left side). So, we'll use the log-log transformation.

Now construct a new column, `log_log_number_of_reviews`, by applying the `log_log` function to the column `number_of_reviews`. Drop the old column, `number_of_reviews`, from the data frame

In [ ]:
# your code here


nyc.describe(include = "all")

In [ ]:
assert("log_log_number_of_reviews" in nyc.columns)
assert("number_of_reviews" not in nyc.columns)
assert(np.isclose(nyc["log_log_number_of_reviews"].mean(), 0.896842))

## Calculated Host Listings Count

Now, let's look at the column `calculated_host_listings_count`. We'll look at the distribution.

In [ ]:
nyc['calculated_host_listings_count'].hist(bins = 50)

Since this is very skewed, let's look at what happens when we apply a log transformtion.

In [ ]:
nyc["calculated_host_listings_count"].apply(np.log).hist(bins = 50)

Still very skewed, so let's try the log log transformation

In [ ]:
nyc["calculated_host_listings_count"].apply(log_log).hist(bins = 50)

Still skewed, and we can tell that the data becoming very discrete is going to be an issue. This might not be the best choice, but let's just stick with applying a log transformation.

In [ ]:
nyc["log_calculated_host_listings_count"] = nyc["calculated_host_listings_count"].apply(np.log)

nyc.drop("calculated_host_listings_count", axis = 1, inplace = True)

nyc.describe(include = "all")

# Task 4: Categorical Features


##Neighborhoods

Next, we'll look at the neighborhood information. We'll start by looking at neighbourhood_group.

In [ ]:
nyc['neighbourhood_group'].value_counts()

There are five classes here, so we'll just one hot encode them.

In [ ]:
nbhd_gp_dummies = pd.get_dummies(nyc["neighbourhood_group"], prefix = "neighbourhood_group")

nyc = nyc.join(nbhd_gp_dummies)
nyc.drop('neighbourhood_group', axis = 1, inplace = True)

nyc.describe()

Now, let's look at the column neighbourhood.

In [ ]:
nyc['neighbourhood'].value_counts()

This one is more complicated. There are some very well represented classes, and some that only occur once. Our strategy will be to one-hot encode the common entries, and ignore the rest.

For our threshold, we will only take entries that occur at least 1000 times. This gives us a reasonable number of entries to one-hot encode.

In [ ]:
common_nbhds = airbnb_df.groupby("neighbourhood").filter(lambda x: len(x) >= 1000)['neighbourhood'].unique()
common_nbhds

Now, we'll group together all entries that occurred fewer than 1000 times into one group, called "Other". This will make it easier to handle only one-hot encoding the common entries.

##💻 Q8: filter neighborhoods function

To do this, we define a function to apply to the function. If the neighborhood is a common one, just return that neighborhood name. Otherwise, return "Other" instead.

In [ ]:
def filter_nbhd(neighborhood):
  pass


#nyc['neighbourhood'] = nyc['neighbourhood'].apply(filter_nbhd)

#nyc['neighbourhood'].value_counts()

In [ ]:
assert(len(nyc['neighbourhood'].unique()) == 12)

Now, you'll one-hot encode the column "neighbourhood", adding new columns with the prefix "neighborhood".

##💻 Q9: one-hot encode neighborhoods

Drop "neighbourhood" and "neighbourhood_Other" from the dataframe.

In [ ]:
# one-hot encode neighbourhood
# your code here


# drop neighbourhood and neighbourhood_Other
# your code here


airbnb_df.describe(include = "all")

In [ ]:
assert("neighbourhood_East Village" in nyc.columns)
assert("neighbourhood_Other" not in nyc.columns)
assert("neighbourhood" not in nyc.columns)

## Room Type

We'll look at room type next.

In [ ]:
nyc['room_type'].value_counts()

##✏ Q10: Reasoning for OHE

We're going to apply one-hot encoding to this feature as it is. Why are we choosing to do the OHE immediately instead of doing additional cleaning on this feature first?

[your answer here]

Let's go ahead and OHE room_type

In [ ]:
room_type_dummies = pd.get_dummies(nyc["room_type"], prefix = "room_type")

nyc = nyc.join(room_type_dummies)
nyc.drop('room_type', axis = 1, inplace = True)

nyc.describe(include = "all")

# Task 5: Next Steps

## Data cleaning relates to model performance

Now that we have gone through our features and made some transformations, we would typically train a model to see how well it performs. Remember, data cleaning is the most impactful thing you can do to improve your model performance.

The following steps would create a model to predict the price of an AirBNB rental based off our transformed feature set.

Steps:
* Selecting our features and target.
* Split the data into testing and training sets.
* Fit selected predictive model to the training data.
* Check appropriate evaluation metric for our training data and for our testing data.
* Plot predicted values again true target values, to visualize how the selected model is performing.

This last step means that we will need to apply transformations to the parameters, that correspond to the transformations we did on the features of the dataset.

An example for implementation with our data set:
* Use selected model trained above to make a predict for `log_price`.
* "Undo" the `log` transformation, and return the predicted price.
* Compare your predictions to the actual prices for the records we pulled out.


##✏ Q11: Next Steps
Even after some data cleaning, suppose that we are not getting very good performance. What are two things you could try next?

Option1:

Option2:

## Finding inspiration in others

The best way to learn how to clean data is to see what other people do!

For this last task, scroll down and select one of the notebooks someone has published on the Kaggle site (linked in Task 1). Note: you may have to dig a bit to see one you like.

Once you have a notebook selected, read through it and answer the following questions.

##✏ Q12: Url of notebook used:

[your answer here]

##✏ Q13: EDA

* Describe something new this workbook has done with EDA.
* What is one reason this may be a good choice for our data.
* What is one reason that this may not be a good choice for our data.
* How does this compare to what we've done so far with the data?


[your answer here]

##💻 Q14: EDA plot
Now, try to replicate this EDA step in the code chunk below. You can copy-paste from the author's work, but you may need to change a few things so it can work on our dataframe `nyc`

In [ ]:
#your code here

##✏ Q15: Data Cleaning


* Describe something new this workbook has done with data cleaning.
* What is one reason this may be a good choice for our data.
* What is one reason that this may not be a good choice for our data.
* How does this compare to what we've done so far with the data?


[your answer here]

##💻 Q16: Data cleaning step
Now, try to replicate this data cleaning step in the code chunk below. You can copy-paste from the author's work, but you may need to change a few things so it can work on our dataframe `nyc`

In [ ]:
#your code here

#Task 6: Reflection

Take a moment to reflect on the assingment



##✏ Q17: Reflection

What did you like about it? What could be improved? Your answers will not affect your overall grade. This feedback will be used to improve future programming assignments.

#Grading

For each of the following accomplishments, there is a breakdown of points which total to 20. The fraction of points earned out of 20 will be multiplied by 5 to get your final score (e.g. 17 points earned will be 17/20 * 5 → 4.25)
* (1pt) Task1 q1: Correctly filters dataset
* (2pt) Task1 q2: Gives 1-2 sectence descriptions for each feature
* (2pt) Task2 q3: Discussed reasons for why all the listed features were dropped
* (1pt) Task2 q4: Discussed reasons for why all the listed features were kept
* (1pt) Task3 q5: Correctly implemented a log function
* (1pt) Task3 q6: Identified the reason for adding 1 to each value
* (1pt) Task3 q7: Correctly implemented a log_log with add-one function
* (1pt) Task4 q8: Correclt implemented a filtering function for neighborhoods
* (1pt) Task4 q9: Replicated the OHE code for neighborhoods
* (1pt) Task4 q10: Discusses why we can do OHE without cleaning
* (2pt) Task5 q11: Gives two scenarios to improve model performance for our airbnb dataset. Must include specific examples from the data.
* (1pt) Task5 q12: Url of inspiration is included
* (1pt) Task5 q13: Discusses new EDA from inspiration notebook
* (1pt) Task5 q14: Replicates (successfully) the EDA plot
* (1pt) Task5 q15: Discusses new data cleaning from inspiration notebook
* (1pt) Task5 q16: Replicates (successfully) the data cleaning
* (1pt) Task6 q17: Thoughtfully reflected on the assignment